# Тестовое задание
выполнил студент группы 9381 Никита Шахин

In [1]:
import numpy as np
from scipy.fft import fft
from pydub import AudioSegment
import more_itertools as mit
import sklearn.cluster
import csv
import random
import copy

Считываем файлы и сохраняем их в массив

In [2]:
signals = []
filenames = []
filename_template = "mp3/{}.mp3"
for i in range(124):
    filename = filename_template.format(i + 1)
    filenames.append(filename)
    audiofile = AudioSegment.from_mp3(filename).set_channels(1)
    signals.append(np.array(audiofile.get_array_of_samples()))


Размер одного интервала

In [3]:
dimension = 256
window_offset = 256

* Каждый трек делим на интервалы.
* В кажом треке для каждого подходящего интервала(интервал, максимум в котором содержит 30% от максимиума самого мощного интервала)
* Для каждого подходящего интервала вычисляется амплитудный спектр
* Находим среднее значение амплитудных спектров

In [27]:
amplitude_spectrums = []
for signal in signals:
    signal_intervals = list(mit.windowed(signal, fillvalue=0, n=dimension, step=window_offset))
    bound = 0
    for i in signal_intervals:
        m = np.max(i)
        if m > bound:
            bound = m
    bound *= 0.15
    signal_intervals = list(filter(lambda i: np.max(i) > bound, signal_intervals))
    signal_intervals = list(map(lambda interval: fft(interval * np.hamming(len(interval))), signal_intervals))
    amplitude_spectrum = [[np.abs(i) for i in interval[:dimension//2]] for interval in signal_intervals]
    a = [0 for i in range(dimension // 2)]
    for i in amplitude_spectrum:
        for j in range(dimension // 2):
            a[j] += i[j]
    for i in range(dimension // 2):
        a[i] /= len(signal_intervals)
    amplitude_spectrums.append(a)

Выполняем алгоритм кластеризации
1. Случайным образом инициализируем центроиды
2. Разбиваем сигналы на кластеры
3. Пересчитываем новые центры(ищем среднее значение в каждом кластере)
Повторяем шаги 2 и 3 пока центроиды меняются

In [36]:
def data_distribution(data, centers):
    dim = len(data[0])
    n = len(data)
    k = len(centers)
    clusters = [[] for i in range(k)]
    for i in range(n):
        min_distance = float('inf')
        situable_cluster = -1
        for j in range(k):
            distance = 0
            for q in range(dim):
                distance += (data[i][q] - centers[j][q]) ** 2
            distance = distance ** 0.5
            if distance < min_distance:
                min_distance = distance
                situable_cluster = j
        clusters[situable_cluster].append(i)
    return clusters

def centers_update(data, clusters, centers):
    dim = len(data[0])
    k = len(centers)   
    for i in range(k): 
        for q in range(dim): 
            if len(clusters[i]) != 0:
                updated_parameter = 0
                for j in range(len(clusters[i])): 
                    updated_parameter += data[clusters[i][j]][q]
                updated_parameter = updated_parameter / len(clusters[i])
                centers[i][q] = updated_parameter

def clusterization(data, k):
    n = len(data)  
    dim = len(data[0])  
    indexes = []
    for i in range(k):
        r = random.randint(0, k)
        while r in indexes:
            r = random.randint(0, k)
        indexes.append(r)   
    centers = [copy.deepcopy(data[indexes[i]]) for i in range(k)]
    clusters = data_distribution(data, centers)
    previous_centers = None        
    while centers != previous_centers:
        previous_centers = copy.deepcopy(centers)
        centers_update(data, clusters, centers)
        clusters = data_distribution(data, centers)
    #Метрика    
    CC = 0    
    for i in range(len(centers)):
        for j in range(len(clusters[i])):
            for q in range(dim):
                CC +=(data[clusters[i][j]][q] - centers[i][q])**2
    CC = CC/5
    print(CC)
    return clusters

res = clusterization(amplitude_spectrums, 5)
ans = []
print(res)
for i in range(len(amplitude_spectrums)):
    for j in range(len(res)):
        if i in res[j]:
            print(j, end=", ")
            ans.append(j)

98779770083.89444
[[18, 23, 29, 32, 37, 40, 45, 48, 49, 50, 51, 68, 70, 72, 80, 84, 87, 89, 91, 104, 109], [12, 24, 30, 34, 35, 36, 38, 39, 42, 43, 44, 46, 47, 52, 55, 63, 64, 69, 71, 73, 74, 75, 76, 77, 78, 81, 83, 93, 95, 99, 107, 116, 123], [1, 4, 7, 10, 27, 41, 53, 54, 56, 57, 58, 60, 61, 62, 66, 79, 82, 85, 86, 88, 90, 92, 94, 103], [0, 2, 3, 5, 6, 8, 9, 11, 14, 16, 17, 19, 25, 26, 28, 31, 33, 59, 65, 67, 96, 97, 98, 100, 101, 106, 110, 111, 112, 113, 114, 115, 118, 119, 120, 121, 122], [13, 15, 20, 21, 22, 102, 105, 108, 117]]
3, 2, 3, 3, 2, 3, 3, 2, 3, 3, 2, 3, 1, 4, 3, 4, 3, 3, 0, 3, 4, 4, 4, 0, 1, 3, 3, 2, 3, 0, 1, 3, 0, 3, 1, 1, 1, 0, 1, 1, 0, 2, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 2, 2, 1, 2, 2, 2, 3, 2, 2, 2, 1, 1, 3, 2, 3, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1, 0, 2, 2, 0, 2, 0, 2, 0, 2, 1, 2, 1, 3, 3, 3, 1, 3, 3, 4, 2, 0, 4, 3, 1, 4, 0, 3, 3, 3, 3, 3, 3, 1, 4, 3, 3, 3, 3, 3, 1, 

In [38]:
with open('test.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(len(ans)):
        spamwriter.writerow(["{}.mp3".format(i+1), ans[i]])